虽然我本地没有多GPU，但是我可以先熟悉这部分的代码，然后修改出来一个多GPU版本的代码。

主要担心的点：
- 模型是直接传递的么
- 还有数据部分，是传递给哪个GPU了
- numpy和torch的转换部分

In [1]:
# 获得设备的GPU个数
import torch

torch.cuda.device_count()

1

看别人代码的时候，看到cuda，所以在想这个和我用的to device有什么区别。
明白了，device可以指定cpu和gpu，但是cuda只能指定gpu。

文章地址： https://zhuanlan.zhihu.com/p/102697821
需要用到的类是：nn.DataParallel

- 首先在前向过程中，你的输入数据会被划分成多个子部分（以下称为副本）送到不同的device中进行计算，而你的模型module是在每个device上进行复制一份，也就是说，输入的batch是会被平均分到每个device中去，但是你的模型module是要拷贝到每个devide中去的，每个模型module只需要处理每个副本即可，当然你要保证你的batch size大于你的gpu个数。然后在反向传播过程中，每个副本的梯度被累加到原始模块中。概括来说就是：DataParallel 会自动帮我们将数据切分 load 到相应 GPU，将模型复制到相应 GPU，进行正向传播计算梯度并汇总。

- 在运行此DataParallel模块之前，并行化模块必须在device_ids [0]上具有其参数和缓冲区。在执行DataParallel之前，会首先把其模型的参数放在device_ids[0]上，一看好像也没有什么毛病，其实有个小坑。我举个例子，服务器是八卡的服务器，刚好前面序号是0的卡被别人占用着，于是你只能用其他的卡来，比如你用2和3号卡，如果你直接指定device_ids=[2, 3]的话会出现模型初始化错误，类似于module没有复制到在device_ids[0]上去。那么你需要在运行train之前需要添加如下两句话指定程序可见的devices，如下：
- os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
- os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3"

- 当你添加这两行代码后，那么device_ids[0]默认的就是第2号卡，你的模型也会初始化在第2号卡上了，而不会占用第0号卡了。这里简单说一下设置上面两行代码后，那么对这个程序而言可见的只有2和3号卡，和其他的卡没有关系，这是物理上的号卡，逻辑上来说其实是对应0和1号卡，即device_ids[0]对应的就是第2号卡，device_ids[1]对应的就是第3号卡。（当然你要保证上面这两行代码需要定义在

- device_ids = [0, 1]
- net = torch.nn.DataParallel(net, device_ids=device_ids)

- 那么在训练过程中，你的优化器同样可以使用nn.DataParallel
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
optimizer = nn.DataParallel(optimizer, device_ids=device_ids)

- 那么使用nn.DataParallel后，事实上DataParallel也是一个Pytorch的nn.Module，那么你的模型和优化器都需要使用.module来得到实际的模型和优化器，如下：
保存模型：
torch.save(net.module.state_dict(), path)
加载模型：
net=nn.DataParallel(Resnet18())
net.load_state_dict(torch.load(path))
net=net.module
优化器使用：
optimizer.step() --> optimizer.module.step()


着手开始修改新的train和model。
在此之前还需要搞明白数据的真正运行，以及如何把test给加入进去。